In [55]:
import math

class KNN:
    def __init__(self):
        self.medians_asds = []
        

    def train(self, data, normal_feature_names, cat_feature_names, sorted_lists, \
                  one_hot_feature_names, feature_names):

        # The features that are not supposed to be changed, need to be converted in float 
        for feature in feature_names:
            if feature in cat_feature_names or feature in one_hot_feature_names:
                continue
            data[feature] = list(map(float, data[feature]))

        for feature_name in normal_feature_names:
            self.normalize_feature(data, feature_name)

        i=0
        for feature_name in cat_feature_names:
            self.cat_to_num(data, feature_name, sorted_lists[i])
            i+=1

        for feature_name in one_hot_feature_names:
            self.one_hot_convert(data, feature_name)


    # converts a categorical feature into a numarical one in an inplace way
    def cat_to_num(self, data, feature_name, sorted_values):
        #values = list(set( data[feature_name] ))
        cat_num={}
        for num in range(len(sorted_values)):
            cat_num[sorted_values[num]] = num

        for i in range(len(data[feature_name])):
            data[feature_name][i] = cat_num[ data[feature_name][i] ]
    

    # replace a categorical feature with multiple features 
    # using one hot encoding in an inplace way
    def one_hot_convert(self, data, feature_name):
        values = list(set( data[feature_name] ))
        cat_num={}
        for num in range(len(values)):
            cat_num[values[num]] = num + 1

        for v in values:
            data[str(feature_name) + '_' + str(cat_num[v])] = []

        for i in range(len(data[feature_name])):
            for v in values:
                data[str(feature_name) + '_' + str(cat_num[v])].append(1 if data[feature_name][i] == v else 0)

        del data[feature_name]


    def normalize_feature(self, data, feature_name):
        median = self.getMedian(data[feature_name])
        asd = self.get_absolute_standard_deviation(data[feature_name], median)
        #print("Median: %f   ASD = %f" % (median, asd))
        self.medians_asds.append((median, asd))
        for i in range(len(data[feature_name])):
            data[feature_name][i] = (data[feature_name][i] - median) / asd


    #returns the median of list l
    def getMedian(self, l):
        if l == []:
            return []
        sorted_l = sorted(l)
        length = len(l)
        if length % 2 == 1: # length of list is odd so return middle element
            return sorted_l[int(((length + 1) / 2) -  1)]
        else: # length of list is even so compute midpoint
            v1 = sorted_l[int(length / 2)]
            v2 =sorted_l[(int(length / 2) - 1)]
            return (v1 + v2) / 2.0
    

    # computes standard deviation
    def get_absolute_standard_deviation(self, l, median):
        return sum([abs(item - median) for item in l]) / len(l)



    def test(self, test_data, data, normal_feature_names, cat_feature_names, \
             sorted_lists, one_hot_feature_names, feature_names, label_name, k=3):
        predictions = []

        # The features that are not supposed to be changed, need to be converted in float 
        for feature in feature_names:
            if feature in cat_feature_names or feature in one_hot_feature_names:
                continue
            test_data[feature] = list(map(float, test_data[feature]))

        for feature_name in normal_feature_names:
            self.normalize_feature(test_data, feature_name)

        i=0
        for feature_name in cat_feature_names:
            self.cat_to_num(test_data, feature_name, sorted_lists[i])
            i+=1

        for feature_name in one_hot_feature_names:
            self.one_hot_convert(test_data, feature_name)

        # make a prediction for every instance
        feature_names = list(test_data.keys())
        feature_names.remove(label_name)
        length = len(test_data[ feature_names[0] ])
        for i in range(length):
            test_instance = [ test_data[feature][i] for feature in feature_names ]
            predictions.append(self.classify(data, feature_names, label_name, test_instance, k))

        return predictions


    # Computes the Manhattan distance.
    def manhattan(self, vector1, vector2):
        return sum(map(lambda v1, v2: abs(v1 - v2), vector1, vector2))



    # compute the cosine similarity of x,y
    def cosine_x_y(self, x,y):
        if x.count(0) == len(x) or y.count(0) == len(y): # All vector's values are zero
            return -2 # Indecating that similarity can't be computed
        numerator = sum([t[0]*t[1] for t in zip(x, y)])
        denominator = sqrt( sum([i * i for i in x]) ) * sqrt( sum([i * i for i in y]) )
        return numerator/denominator



    def classify(self, data, feature_names, label_name, test_instance, k):
        neighbors = []
        for i in range( len(data[label_name]) ):
            instance = [ data[feature][i] for feature in feature_names ]
            neighbors.append( (self.manhattan(instance, test_instance), i) )

        neighbors_labels = [ data[label_name][item[1]] for item in sorted(neighbors)[:k] ]
        return max(set(neighbors_labels), key=neighbors_labels.count)


    

In [51]:
# Replace the values of a list with a new item if they exist in o_items
def replace_items(l, o_items, n_item):
    return [n_item if item in o_items else item for item in l]

# fill missing values in missing_feature with most frequent value per class
def fill_missing(data, label_name, missing_feature, missing_value):
    best_filling = {}
    for klass in list(set(data[label_name])):
        best_filling[klass] = []

    for i in range(len(data[label_name])):
        best_filling[data[label_name][i]].append(data[missing_feature][i])

    for klass, values in best_filling.items():
        best_filling[klass] = max(set(values), key=values.count)

    for i in range(len(data[missing_feature])):
        if data[missing_feature][i] == missing_value:
            data[missing_feature][i] = best_filling[data[label_name][i]]

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pprint import pprint

# breast-cancer-wisconsin dataset

## One Run

In [40]:
df = pd.read_csv('./breast-cancer-wisconsin.data', header=None)
# drop rows with missing values, missing = ?
df = df.replace("?", np.nan)
df = df.dropna()
# organize data into input and output
X = df.drop(columns=10)
y = df[10]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33) 
d_demo = pd.concat([X_train, y_train], axis=1).to_dict(orient='list')
d_test = pd.concat([X_test, y_test], axis=1).to_dict(orient='list')

feature_names = list(d_demo.keys())
label_name = feature_names[-1]
feature_names.remove(label_name)
feature_names.remove(0)
normal_feature_names = feature_names
cat_feature_names = []
one_hot_feature_names = []


classifier = KNN()
classifier.train(d_demo, normal_feature_names, cat_feature_names, [], one_hot_feature_names, feature_names)
my_pred = classifier.test(d_test, d_demo, normal_feature_names, cat_feature_names, [], 
                   one_hot_feature_names, feature_names, label_name, k=3)
accuracy_score(y_test, my_pred)

0.6017699115044248

## 50 runs (10 * 5 folds)

In [41]:
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold


df = pd.read_csv('./breast-cancer-wisconsin.data', header=None)
# drop rows with missing values, missing = ?
df = df.replace("?", np.nan)
df = df.dropna()
df = df.reset_index()
target = df[10]
skf = KFold(n_splits=5, shuffle=True)
accs = []
for run in range(10):
    print("================Run {}================".format(run))
    fold_no = 1
    for train_index, test_index in skf.split(df, target):
        train = df.loc[train_index,:]
        test = df.loc[test_index,:]

        X = train.drop(columns=10)
        y = train[10]
        d_demo = pd.concat([X, y], axis=1).to_dict(orient='list')

        feature_names = list(d_demo.keys())
        label_name = feature_names[-1]
        feature_names.remove(label_name)
        feature_names.remove(0)
        
        X_test = test.drop(columns=10)
        y_test = test[10]
        d_test = pd.concat([X_test, y_test], axis=1).to_dict(orient='list')
        

        normal_feature_names = feature_names
        cat_feature_names = []
        sorted_lists = []
        one_hot_feature_names = []
        
        classifier = KNN()
        classifier.train(d_demo, normal_feature_names, cat_feature_names, [], one_hot_feature_names, feature_names)
        my_pred = classifier.test(d_test, d_demo, normal_feature_names, cat_feature_names, [], 
                           one_hot_feature_names, feature_names, label_name, k=3)     

        print("fold: ", fold_no, "===>", "accuracy: ", accuracy_score(y_test, my_pred))
        accs.append(accuracy_score(y_test, my_pred))
        fold_no += 1


================Run 0================
fold:  1 ===> accuracy:  0.6204379562043796
fold:  2 ===> accuracy:  0.6277372262773723
fold:  3 ===> accuracy:  0.5985401459854015
fold:  4 ===> accuracy:  0.6323529411764706
fold:  5 ===> accuracy:  0.6397058823529411
================Run 1================
fold:  1 ===> accuracy:  0.6058394160583942
fold:  2 ===> accuracy:  0.635036496350365
fold:  3 ===> accuracy:  0.5693430656934306
fold:  4 ===> accuracy:  0.6397058823529411
fold:  5 ===> accuracy:  0.5808823529411765
================Run 2================
fold:  1 ===> accuracy:  0.583941605839416
fold:  2 ===> accuracy:  0.635036496350365
fold:  3 ===> accuracy:  0.5693430656934306
fold:  4 ===> accuracy:  0.6323529411764706
fold:  5 ===> accuracy:  0.6691176470588235
================Run 3================
fold:  1 ===> accuracy:  0.6423357664233577
fold:  2 ===> accuracy:  0.5766423357664233
fold:  3 ===> accuracy:  0.5547445255474452
fold:  4 ===> accuracy:  0.625
fold:  5 ===> accuracy:  0.6

In [42]:
len(accs)

50

In [43]:
import statistics
print("Standard deviation: ", statistics.stdev(accs) )
print("Mean: ", statistics.mean(accs) )

Standard deviation:  0.03222847194400608
Mean:  0.6139297981966509


# car dataset

## One Run

In [45]:
df = pd.read_csv('./car.data', header=None)
# drop rows with missing values, missing = ?
df = df.replace("?", np.nan)
df = df.dropna()
# organize data into input and output
X = df.drop(columns=6)
y = df[6]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
d_demo = pd.concat([X_train, y_train], axis=1).to_dict(orient='list')
d_demo[2] = replace_items(d_demo[2], '5more', '5')
d_demo[3] = replace_items(d_demo[3], 'more', '3')
feature_names = list(d_demo.keys())
label_name = feature_names[-1]
feature_names.remove(label_name)
normal_feature_names = [2, 3]
cat_feature_names = [0, 1, 4, 5]
sorted_lists = [['low', 'med', 'high', 'vhigh'], 
                ['low', 'med', 'high', 'vhigh'], 
                ['small', 'med', 'big'], 
                ['low', 'med', 'high']]
one_hot_feature_names = []
d_test = pd.concat([X_test, y_test], axis=1).to_dict(orient='list')
d_test[2] = replace_items(d_test[2], '5more', '5')
d_test[3] = replace_items(d_test[3], 'more', '3')

classifier = KNN()
classifier.train(d_demo, normal_feature_names, cat_feature_names, sorted_lists, one_hot_feature_names, feature_names)
my_pred = classifier.test(d_test, d_demo, normal_feature_names, cat_feature_names, sorted_lists, 
                   one_hot_feature_names, feature_names, label_name, k=3)
accuracy_score(y_test, my_pred)

0.9132947976878613

## 50 runs (10 * 5 folds)

In [46]:
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold


df = pd.read_csv('./car.data', header=None)
# drop rows with missing values, missing = ?
df = df.replace("?", np.nan)
df = df.dropna()
target = df[6]
skf = KFold(n_splits=5, shuffle=True)
accs = []
for run in range(10):
    print("================Run {}================".format(run))
    fold_no = 1
    for train_index, test_index in skf.split(df, target):
        train = df.loc[train_index,:]
        test = df.loc[test_index,:]

        X = train.drop(columns=6)
        y = train[6]
        d_demo = pd.concat([X, y], axis=1).to_dict(orient='list')
        d_demo[2] = replace_items(d_demo[2], '5more', '5')
        d_demo[3] = replace_items(d_demo[3], 'more', '3')

        feature_names = list(d_demo.keys())
        label_name = feature_names[-1]
        feature_names.remove(label_name)
        
        X_test = test.drop(columns=6)
        y_test = test[6]
        d_test = pd.concat([X_test, y_test], axis=1).to_dict(orient='list')
        d_test[2] = replace_items(d_test[2], '5more', '5')
        d_test[3] = replace_items(d_test[3], 'more', '3')
        
        normal_feature_names = [2, 3]
        cat_feature_names = [0, 1, 4, 5]
        sorted_lists = [['low', 'med', 'high', 'vhigh'], 
                ['low', 'med', 'high', 'vhigh'], 
                ['small', 'med', 'big'], 
                ['low', 'med', 'high']]        
        one_hot_feature_names = []
        
        classifier = KNN()
        classifier.train(d_demo, normal_feature_names, cat_feature_names, sorted_lists, \
                         one_hot_feature_names, feature_names)
        my_pred = classifier.test(d_test, d_demo, normal_feature_names, cat_feature_names, \
                                  sorted_lists, one_hot_feature_names, feature_names, \
                                  label_name, k=3)       

        print("fold: ", fold_no, "===>", "accuracy: ", accuracy_score(y_test, my_pred))
        accs.append(accuracy_score(y_test, my_pred))
        fold_no += 1


================Run 0================
fold:  1 ===> accuracy:  0.8786127167630058
fold:  2 ===> accuracy:  0.846820809248555
fold:  3 ===> accuracy:  0.8901734104046243
fold:  4 ===> accuracy:  0.9159420289855073
fold:  5 ===> accuracy:  0.9014492753623189
================Run 1================
fold:  1 ===> accuracy:  0.9248554913294798
fold:  2 ===> accuracy:  0.9075144508670521
fold:  3 ===> accuracy:  0.8757225433526011
fold:  4 ===> accuracy:  0.8666666666666667
fold:  5 ===> accuracy:  0.8695652173913043
================Run 2================
fold:  1 ===> accuracy:  0.8959537572254336
fold:  2 ===> accuracy:  0.9017341040462428
fold:  3 ===> accuracy:  0.8988439306358381
fold:  4 ===> accuracy:  0.8956521739130435
fold:  5 ===> accuracy:  0.8782608695652174
================Run 3================
fold:  1 ===> accuracy:  0.9017341040462428
fold:  2 ===> accuracy:  0.8901734104046243
fold:  3 ===> accuracy:  0.8815028901734104
fold:  4 ===> accuracy:  0.8927536231884058
fold:  5 ===>

In [47]:
len(accs)

50

In [48]:
import statistics
print("Standard deviation: ", statistics.stdev(accs) )
print("Mean: ", statistics.mean(accs) )

Standard deviation:  0.018918009892190116
Mean:  0.8883143168300243


# mushroom dataset

## One Run

In [56]:
df = pd.read_csv('./mushroom.data', header=None)
# organize data into input and output
X = df.drop(columns=0)
y = df[0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
d_demo = pd.concat([X_train, y_train], axis=1).to_dict(orient='list')
feature_names = list(d_demo.keys())
label_name = feature_names[-1]
feature_names.remove(label_name)
feature_names.remove(16)
normal_feature_names = []
cat_feature_names = []
sorted_lists = []
one_hot_feature_names = feature_names
#d_test = X_test.to_dict(orient="list")
d_test = pd.concat([X_test, y_test], axis=1).to_dict(orient='list')
fill_missing(d_demo, label_name, 11, '?')
fill_missing(d_test, label_name, 11, '?')
del d_demo[16]
del d_test[16]

classifier = KNN()
classifier.train(d_demo, normal_feature_names, cat_feature_names, sorted_lists, one_hot_feature_names, feature_names)
my_pred = classifier.test(d_test, d_demo, normal_feature_names, cat_feature_names, sorted_lists, 
                   one_hot_feature_names, feature_names, label_name, k=3)
accuracy_score(y_test, my_pred)

1.0

## 50 runs (10 * 5 folds)

**It takes a long time**

# ecoli dataset

## One Run

In [57]:
df = pd.read_csv('./ecoli.data', header=None)
# organize data into input and output
X = df.drop(columns=8)
y = df[8]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
d_demo = pd.concat([X_train, y_train], axis=1).to_dict(orient='list')
feature_names = list(d_demo.keys())
label_name = feature_names[-1]
feature_names.remove(label_name)
#d_test = X_test.to_dict(orient="list")
d_test = pd.concat([X_test, y_test], axis=1).to_dict(orient='list')
del d_demo[0]
del d_test[0]
feature_names.remove(0)
normal_feature_names = []
#cat_feature_names = [ feature_names[i] for i in [3,4] ]
#sorted_lists = [['0.48', '1.0'], 
#                [0.5, '1.0']]
cat_feature_names = []
sorted_lists = []
one_hot_feature_names = []

classifier = KNN()
classifier.train(d_demo, normal_feature_names, cat_feature_names, sorted_lists, one_hot_feature_names, feature_names)
my_pred = classifier.test(d_test, d_demo, normal_feature_names, cat_feature_names, sorted_lists, 
                   one_hot_feature_names, feature_names, label_name, k=3)
accuracy_score(y_test, my_pred)

0.8828828828828829

## 50 runs (10 * 5 folds)

In [58]:
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold


df = pd.read_csv('./ecoli.data', header=None)
target = df[0]
skf = KFold(n_splits=5, shuffle=True)
accs = []
for run in range(10):
    print("================Run {}================".format(run))
    fold_no = 1
    for train_index, test_index in skf.split(df, target):
        train = df.loc[train_index,:]
        test = df.loc[test_index,:]

        X = train.drop(columns=8)
        y = train[8]
        d_demo = pd.concat([X, y], axis=1).to_dict(orient='list')

        feature_names = list(d_demo.keys())
        label_name = feature_names[-1]
        feature_names.remove(label_name)
        
        X_test = test.drop(columns=8)
        y_test = test[8]
        d_test = pd.concat([X_test, y_test], axis=1).to_dict(orient='list')
        
        del d_demo[0]
        del d_test[0]
        feature_names.remove(0)
        normal_feature_names = []
        #cat_feature_names = [ feature_names[i] for i in [3,4] ]
        #sorted_lists = [['0.48', '1.0'], 
        #                [0.5, '1.0']]
        cat_feature_names = []
        sorted_lists = []
        one_hot_feature_names = []
        
        classifier = KNN()
        classifier.train(d_demo, normal_feature_names, cat_feature_names, sorted_lists, one_hot_feature_names, feature_names)
        my_pred = classifier.test(d_test, d_demo, normal_feature_names, cat_feature_names, sorted_lists, 
                           one_hot_feature_names, feature_names, label_name, k=3)       

        print("fold: ", fold_no, "===>", "accuracy: ", accuracy_score(y_test, my_pred))
        accs.append(accuracy_score(y_test, my_pred))
        fold_no += 1


================Run 0================
fold:  1 ===> accuracy:  0.8235294117647058
fold:  2 ===> accuracy:  0.8656716417910447
fold:  3 ===> accuracy:  0.8208955223880597
fold:  4 ===> accuracy:  0.835820895522388
fold:  5 ===> accuracy:  0.8955223880597015
================Run 1================
fold:  1 ===> accuracy:  0.8235294117647058
fold:  2 ===> accuracy:  0.8805970149253731
fold:  3 ===> accuracy:  0.7910447761194029
fold:  4 ===> accuracy:  0.8656716417910447
fold:  5 ===> accuracy:  0.9104477611940298
================Run 2================
fold:  1 ===> accuracy:  0.8676470588235294
fold:  2 ===> accuracy:  0.8059701492537313
fold:  3 ===> accuracy:  0.8955223880597015
fold:  4 ===> accuracy:  0.8059701492537313
fold:  5 ===> accuracy:  0.8507462686567164
================Run 3================
fold:  1 ===> accuracy:  0.8970588235294118
fold:  2 ===> accuracy:  0.835820895522388
fold:  3 ===> accuracy:  0.7164179104477612
fold:  4 ===> accuracy:  0.8656716417910447
fold:  5 ===> 

In [62]:
len(accs)

50

In [63]:
import statistics
print("Standard deviation: ", statistics.stdev(accs) )
print("Mean: ", statistics.mean(accs) )

Standard deviation:  0.04997225500068545
Mean:  0.84486391571554


# letter-recognition dataset

## One Run

In [64]:
df = pd.read_csv('./letter-recognition.data', header=None)
# organize data into input and output
X = df.drop(columns=0)
y = df[0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
d_demo = pd.concat([X_train, y_train], axis=1).to_dict(orient='list')
feature_names = list(d_demo.keys())
label_name = feature_names[-1]
feature_names.remove(label_name)
normal_feature_names = feature_names
cat_feature_names = []
one_hot_feature_names = []
sorted_lists = []
d_test = pd.concat([X_test, y_test], axis=1).to_dict(orient='list')

classifier = KNN()
classifier.train(d_demo, normal_feature_names, cat_feature_names, sorted_lists, one_hot_feature_names, feature_names)
my_pred = classifier.test(d_test, d_demo, normal_feature_names, cat_feature_names, sorted_lists, 
                   one_hot_feature_names, feature_names, label_name, k=3)

In [65]:
accuracy_score(y_test, my_pred)

0.9443939393939393

## 50 runs (10 * 5 folds)

**It takes a long time**